<a href="https://colab.research.google.com/github/gopixiedust/Algo-Bazziiii/blob/main/Stonks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
% pip install jugaad-data

     |████████████████████████████████| 115 kB 6.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
returnVal=0

In [ ]:
from datetime import  date,timedelta
from jugaad_data.nse import  stock_df

# Download as pandas dataframe
df = stock_df(symbol="infy", from_date=date.today()-timedelta(days=200),
            to_date=date.today(), series="EQ")
# df.head(10)
df.drop(['SERIES', 
       'VWAP', '52W H', '52W L', 'VOLUME', 'VALUE', 'NO OF TRADES', 'SYMBOL'],inplace=True,axis=1)
df=df[::-1]
# df=df[0:101]
# df=df.reset_index()
df=df.reset_index()
print(df.tail(10))


     index       DATE     OPEN     HIGH      LOW  PREV. CLOSE      LTP    CLOSE
126      9 2021-08-04  1669.10  1679.90  1639.05      1655.20  1648.55  1649.25
127      8 2021-08-05  1649.50  1666.00  1647.05      1649.25  1652.00  1653.55
128      7 2021-08-06  1657.00  1660.65  1644.00      1653.55  1652.10  1650.20
129      6 2021-08-09  1661.00  1667.45  1646.40      1650.20  1664.40  1663.30
130      5 2021-08-10  1668.00  1680.00  1661.05      1663.30  1674.15  1677.25
131      4 2021-08-11  1674.80  1684.00  1668.00      1677.25  1675.10  1677.55
132      3 2021-08-12  1679.15  1696.50  1674.00      1677.55  1684.95  1689.60
133      2 2021-08-13  1697.00  1722.50  1691.30      1689.60  1713.95  1712.20
134      1 2021-08-16  1707.70  1719.95  1690.10      1712.20  1702.95  1704.40
135      0 2021-08-17  1703.90  1748.90  1686.55      1704.40  1737.40  1741.65


In [ ]:
# retreiving and storing data - changed source of data, this one is old
import numpy as np
import pandas as pd

pd.options.display.max_columns = None

# import requests

# key = 'ZZW44UFCBUQQUQLP'
# symbol = 'INFY'

# url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={key}'
# r = requests.get(url)
# data = r.json()['Time Series (Daily)']

# df = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close'])

# for day in data.keys():
#   df = df.append({'Date': day, 'Open': data[day]['1. open'], 'High': data[day]['2. high'], 'Low': data[day]['3. low'], 'Close': data[day]['4. close']}, ignore_index=True)
# df['Open'] = pd.to_numeric(df['Open'])
# df['High'] = pd.to_numeric(df['High'])
# df['Low'] = pd.to_numeric(df['Low'])
# df['Close'] = pd.to_numeric(df['Close'])

# df = df[::-1]
# df = df.reset_index()
# del df['index']

# df

In [ ]:
# calculating EMAs - works
emadf = pd.DataFrame({'CLOSE':df['CLOSE']})

emadf['EMA100'] = emadf['CLOSE'].ewm(span=100, min_periods=0, adjust=True).mean()
emadf['EMA50'] = emadf['CLOSE'].ewm(span=50, min_periods=0, adjust=True).mean()

ema100 = emadf['EMA100'].iloc[-1]
ema50 = emadf['EMA50'].iloc[-1]

if (ema50>ema100):
  returnVal+=1

print(returnVal)
print('ema100: ', ema100)
print('ema50: ', ema50)

1
ema100:  1516.3826396130492
ema50:  1579.678925676566


In [ ]:
# calculating RSI - works
rsidf = pd.DataFrame({'CLOSE':df['CLOSE']})

def rma(x, n, y0):
    a = (n-1) / n
    ak = a**np.arange(len(x)-1, -1, -1)
    return np.r_[np.full(n, np.nan), y0, np.cumsum(ak * x) / ak / n + y0 * a**np.arange(1, len(x)+1)]

n=14

rsidf['change'] = rsidf['CLOSE'].diff()
rsidf['change'][0] = 0
rsidf['gain'] = rsidf.change.mask(rsidf.change < 0, 0.0)
rsidf['loss'] = -rsidf.change.mask(rsidf.change > 0, -0.0)
rsidf['avg_gain'] = rma(rsidf.gain[n+1:].to_numpy(), n, np.nansum(rsidf.gain.to_numpy()[:n+1])/n)
rsidf['avg_loss'] = rma(rsidf.loss[n+1:].to_numpy(), n, np.nansum(rsidf.loss.to_numpy()[:n+1])/n)
rsidf['rs'] = rsidf.avg_gain / rsidf.avg_loss
rsidf['rsi_14'] = 100 - (100 / (1 + rsidf.rs))

rsi = rsidf['rsi_14'][99]
print('rsi: ', rsi)

rsi:  84.41767989507962


In [ ]:
# calculating stochastic oscillator - works

sodf = pd.DataFrame(df[-14::]) # get latest 14 days

c = sodf['CLOSE'].iloc[-1]
l = sodf['LOW'].min()
h = sodf['HIGH'].max()

so = (c-l)/(h-l) * 100

print('so: ', so)

so:  95.12768817204301


In [ ]:
# redundant (?)

delta = df['CLOSE'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=True).mean()
ema_down = down.ewm(com=13, adjust=True).mean()
rs = ema_up/ema_down

df['RSI'] = 100 - (100/(1 + rs))
df.RSI

0             NaN
1      100.000000
2      100.000000
3      100.000000
4       87.998576
          ...    
131     76.095725
132     77.988354
133     81.023093
134     77.073565
135     81.669104
Name: RSI, Length: 136, dtype: float64

In [ ]:
if (ema50>ema100):
  returnVal+=1
if (rsi<30):
  returnVal+=1
if (so<20):
  returnVal+=1
print(returnVal);

4
